### SETUP & CONSTANTS

In [1]:
import re
import os
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
import pyspark.sql.functions as F
from pyspark.sql import Window
from functools import reduce

# CONSTANTS
INPUT_PATH = './notebooks/csv/'
TABLE_NAMES = [
    'Purchases',
    'Sales',
    'BegInv',
    'EndInv',
    'PurchasePrices',
    'InvoicePurchases',
]

In [2]:
def unique(df, group_by, select_cols,min=1):
    return df.groupBy(*group_by).agg(F.collect_set(F.struct(*select_cols)).alias('objs')).where(F.array_size('objs')>min)

In [3]:
spark= (
    SparkSession.builder.appName("transform")
    .getOrCreate()
)
existing_csv = os.listdir(INPUT_PATH)
_df_raw={}


for file in existing_csv:
    filename=rf'{INPUT_PATH}/{file}'
    table_name = re.sub(r'FINAL|Dec|\d+|\.csv', '', file)
    if table_name not in TABLE_NAMES:
        continue

    _df_raw[table_name]=spark.read.format("csv").option("header",True).load(filename)
    print(f'{table_name}:')
    _df_raw[table_name].limit(2).show()



PurchasePrices:
+-----+--------------------+-----+-----+------+--------------+-------------+------------+--------------------+
|Brand|         Description|Price| Size|Volume|Classification|PurchasePrice|VendorNumber|          VendorName|
+-----+--------------------+-----+-----+------+--------------+-------------+------------+--------------------+
|   58|Gekkeikan Black &...|12.99|750mL|   750|             1|         9.28|        8320|SHAW ROSS INT L I...|
|   62|Herradura Silver ...|36.99|750mL|   750|             1|        28.67|        1128|BROWN-FORMAN CORP...|
+-----+--------------------+-----+-----+------+--------------+-------------+------------+--------------------+

BegInv:
+-----------------+-----+------------+-----+--------------------+-----+------+-----+----------+
|      InventoryId|Store|        City|Brand|         Description| Size|onHand|Price| startDate|
+-----------------+-----+------------+-----+--------------------+-----+------+-----+----------+
|1_HARDERSFIELD_58|  

### SUMMARY

### SIMPLE TABLES

#### _df_product

In [4]:
product_cols = ['Brand', 'Description', 'Size']

_df_product_raw = (
        reduce(
        DataFrame.unionByName,
        [
            _df_raw['Purchases'].select(*product_cols),
            _df_raw['Sales'].select(*product_cols),
            _df_raw['BegInv'].select(*product_cols),
            _df_raw['EndInv'].select(*product_cols),
        ]
    )
    .select(F.col('Brand').cast('int').alias('Brand'),'Description','Size')
)
_df_product_raw.cache()

_df_product = (
    _df_product_raw
    .groupBy('Brand')
    .agg(
        F.collect_set('Description').alias('alt_names'),
        F.first('Size').alias('Size')
    )
    .withColumn('Description', F.expr("alt_names[0]")) # just take the first description available
    .withColumn('alt_descriptions', F.expr("slice(alt_names, 2, size(alt_names) - 1)")) # store alternate names though
    .select(
        F.row_number().over(Window.orderBy('Brand')).alias('id'),
        'Brand',
        'Description',
        'Size',
        'alt_descriptions',
    )
)
_df_product.cache()

DataFrame[id: int, Brand: int, Description: string, Size: string, alt_descriptions: array<string>]

#### _df_vendor

In [5]:
vendor_cols=['VendorNumber', 'VendorName']
_df_vendor = (
    reduce(
        DataFrame.unionByName,
        [
            _df_raw['Purchases'].select(*vendor_cols),
            _df_raw['InvoicePurchases'].select(*vendor_cols),
            _df_raw['PurchasePrices'].select(*vendor_cols),
            _df_raw['Sales'].withColumnRenamed('VendorNo','VendorNumber').select(*vendor_cols), # may not be necessary
        ]
    )
    .groupBy(F.col('VendorNumber').cast('int').alias('VendorNumber'))
    .agg(
        F.collect_set('VendorName').alias('alt_names')
    )
    .withColumn('VendorName', F.expr("alt_names[0]"))
    .withColumn('alt_vendor_names', F.expr("slice(alt_names, 2, size(alt_names) - 1)"))
    .select(
        F.row_number().over(Window.orderBy('VendorNumber')).alias('id'),
        'VendorNumber',
        'VendorName',
        'alt_vendor_names',
    )
)

#### _df_city

In [6]:
_df_inventory_union = _df_raw['BegInv'].unionByName(_df_raw['BegInv'],allowMissingColumns=True)

_df_city = (
    _df_inventory_union
    .select('City')
    .distinct()
    .select(
        F.row_number().over(Window.orderBy('City')).alias('id'),
        'City'
    )
)

#### _df_store

In [7]:
_df_store = (
    _df_inventory_union
    .join(_df_city.select('id','City').alias('c'), ['City'], 'left')
    .select(F.col('Store').cast('int').alias('Store'),F.col('c.id').alias('city_id'))
    .distinct()
    .select(
        F.row_number().over(Window.orderBy('Store','city_id')).alias('id'),
        F.col('Store').alias('Store'),
        'city_id'
    )
)

#### _df_inventory

In [8]:
inventory_cols = ['InventoryID', 'Store', 'Brand']
_df_inventory_raw = (
    reduce(
        DataFrame.unionByName,
        [
            _df_raw['Purchases'].select(*inventory_cols),
            _df_raw['Sales'].select(*inventory_cols),
            _df_raw['BegInv'].select(*inventory_cols),
            _df_raw['EndInv'].select(*inventory_cols),
        ]
    ).distinct()
)
_df_inventory_raw.cache()

DataFrame[InventoryID: string, Store: string, Brand: string]

In [9]:
shared_inventory_cols = ['InventoryId','Store','Brand']
_df_inventory = (
    _df_inventory_raw
    .join(_df_raw['BegInv'].alias('start'), shared_inventory_cols,'left')
    .join(_df_raw['EndInv'].alias('end'), shared_inventory_cols,'left')
    .withColumn('Brand', F.col('Brand').cast('int'))
    .join(_df_store.select(F.col('id').alias('store_id'),'Store').alias('s'), ['Store'], 'left')
    .join(_df_product.select(F.col('id').alias('product_id'),'Brand').alias('p'), ['Brand'], 'left')
    .select(
        F.row_number().over(Window.orderBy('InventoryId','store_id','product_id')).alias('id'),
        'InventoryId','store_id','product_id', # PK
        # F.struct('start.price', F.col('start.startDate').alias('date'), 'start.onHand').alias('start'),
        # F.struct('end.price', F.col('end.endDate').alias('date'), 'end.onHand').alias('end'),
        F.col('start.price').alias('start_price'),
        F.col('start.startDate').alias('start_date'),
        F.col('start.onHand').alias('start_onHand'),
        F.col('end.price').alias('end_price'),
        F.col('end.endDate').alias('end_date'),
        F.col('end.onHand').alias('end_onHand'),
    )
)



#### _df_purchase_order

In [10]:
purchase_order_cols = ['PONumber', 'PODate', 'VendorNumber', 'InvoiceDate', 'PayDate']
_df_purchase_order = (
    _df_raw['Purchases'].select(*purchase_order_cols)
    .join(
        _df_raw['InvoicePurchases'].select(*purchase_order_cols),
        purchase_order_cols,
        'outer'
    )
    .withColumn('VendorNumber', F.col('VendorNumber').cast('int'))
    .join(
        _df_vendor.select(F.col('id').alias('vendor_id'), 'VendorNumber'),
        ['VendorNumber'],
        'left'
    )
    .select(
        F.col('PONumber').cast('int').alias('PONumber'),
        F.col('PODate').cast('date').alias('PODate'),
        'vendor_id',
        F.col('InvoiceDate').cast('date').alias('InvoiceDate'),
        F.col('PayDate').cast('date').alias('PayDate'),
    )
    .distinct()
    .select(
        F.row_number().over(Window.orderBy('PONumber', 'PODate', 'vendor_id', 'InvoiceDate', 'PayDate')).alias('id'),
        'PONumber', 
        'PODate', 
        'vendor_id',
        'InvoiceDate',
        'PayDate',
    )
)
_df_purchase_order.orderBy('id').show()

+---+--------+----------+---------+-----------+----------+
| id|PONumber|    PODate|vendor_id|InvoiceDate|   PayDate|
+---+--------+----------+---------+-----------+----------+
|  1|    8106|2015-12-20|        8| 2016-01-12|2016-02-05|
|  2|    8107|2015-12-20|      119| 2016-01-05|2016-02-10|
|  3|    8108|2015-12-20|       18| 2016-01-11|2016-02-10|
|  4|    8109|2015-12-20|       22| 2016-01-12|2016-02-11|
|  5|    8110|2015-12-20|       37| 2016-01-09|2016-02-19|
|  6|    8111|2015-12-20|      103| 2016-01-10|2016-02-04|
|  7|    8112|2015-12-20|       49| 2016-01-11|2016-02-06|
|  8|    8113|2015-12-20|       77| 2016-01-08|2016-02-07|
|  9|    8114|2015-12-20|       64| 2016-01-08|2016-02-14|
| 10|    8115|2015-12-20|       98| 2016-01-10|2016-02-10|
| 11|    8116|2015-12-20|       86| 2016-01-06|2016-02-07|
| 12|    8117|2015-12-20|      114| 2016-01-11|2016-02-09|
| 13|    8118|2015-12-20|       97| 2016-01-06|2016-02-20|
| 14|    8119|2015-12-20|       78| 2016-01-11|2016-02-1

#### _df_invoice

In [11]:
_df_invoice = (
    _df_raw['InvoicePurchases']
    .join(_df_vendor.select(F.col('id').alias('vendor_id'),'VendorNumber'), ['VendorNumber'], 'left')
    .join(_df_purchase_order.withColumnRenamed('id','purchase_order_id'),['PONumber', 'PODate', 'vendor_id', 'InvoiceDate', 'PayDate'])
    .select(
        F.row_number().over(Window.orderBy('purchase_order_id')).alias('id'),
        'purchase_order_id',
        F.col('Quantity').cast('int').alias('Quantity'),
        F.col('Dollars').cast('float').alias('Dollars'),
        F.col('Freight').cast('float').alias('Freight'),
    )
)

#### _df_purchase

In [12]:
_df_purchase = (
    _df_raw['Purchases']
    .join(_df_vendor.select(F.col('id').alias('vendor_id'),'VendorNumber'), ['VendorNumber'], 'left')
    .join(_df_inventory.select(F.col('id').alias('inventory_id'),'InventoryId',),['InventoryId'],'left')
    .join(_df_purchase_order.withColumnRenamed('id','purchase_order_id'),['PONumber', 'PODate', 'vendor_id', 'InvoiceDate', 'PayDate'])
    .select(
        'purchase_order_id',
        'inventory_id',
        F.col('ReceivingDate').cast('date').alias('ReceivingDate'),
        F.col('PurchasePrice').cast('float').alias('PurchasePrice'),
        F.col('Quantity').cast('int').alias('Quantity'),
        F.col('Dollars').cast('float').alias('Dollars'),
    )
    .select(
        F.row_number().over(Window.orderBy('purchase_order_id','inventory_id')).alias('id'),
        'purchase_order_id',
        'inventory_id',
        'ReceivingDate',
        'PurchasePrice',
        'Quantity',
        'Dollars',
    )
    
)
_df_purchase.cache()

DataFrame[id: int, purchase_order_id: int, inventory_id: int, ReceivingDate: date, PurchasePrice: float, Quantity: int, Dollars: float]

#### _df_sale

In [13]:
_df_sale = (
    _df_raw['Sales']
    .withColumn('VendorNumber', F.col('VendorNo').cast('int'))
    .join(_df_inventory.select(F.col('id').alias('inventory_id'),'InventoryId'), ['InventoryId'], 'left')
    .join(_df_vendor.select(F.col('id').alias('vendor_id'),'VendorNumber'), ['VendorNumber'], 'left')
    .select(
        'inventory_id',
        'vendor_id',
        F.col('SalesDate').cast('date').alias('SalesDate'),
        F.col('SalesQuantity').cast('int').alias('Quantity'),
        F.col('SalesDollars').cast('float').alias('Dollars'),
        F.col('SalesPrice').cast('float').alias('SalesPrice'),
        F.col('ExciseTax').cast('float').alias('ExciseTax'),
    )
    .select(
        F.row_number().over(Window.orderBy('inventory_id', 'vendor_id', 'SalesDate')).alias('id'),
        'inventory_id',
        'vendor_id',
        'SalesDate',
        'Quantity',
        'Dollars',
        'SalesPrice',
        'ExciseTax',
    )
)

### EXTRA

### ANOMALIES

#### Vendors names aren't normalized

In [14]:
# unique(_df_raw['Purchases'], ['VendorNumber'], ['VendorName']).show(10,truncate=False)

#### Brand=Product. Description and size colums aren't normalized though

In [15]:
# unique(_df_product_raw, ['Brand'], ['Description'],1).show(10, truncate=False)
# unique(_df_product_raw, ['Brand'], ['Size'],1).show(10, truncate=False)

In [16]:
# purchase_order_cols = ['PONumber', 'PODate', 'VendorNumber']
# _df_purchase_orders = (
#     _df_raw['Purchases'].select(*purchase_order_cols,F.lit(True).alias('is_purchase'))
#     .join(
#         _df_raw['InvoicePurchases'].select(*purchase_order_cols,F.lit(True).alias('is_invoice')),
#         purchase_order_cols,
#         'outer'
#     )
# )
# _df_purchase_orders.show()

#### Purchase Orders data is inconsistent. PONumber is not an identifier

In [17]:
# common_cols = [
#     'PONumber', 
# ]
# _df_temp = (
#     _df_raw['Purchases']
#     .withColumn('purchase', F.lit(True))
#     .groupBy('PONumber', 'PODate', 'VendorNumber','InvoiceDate','PayDate', 'purchase').agg(F.sum('Quantity').cast('int').alias('Quantity'), F.round(F.sum('Dollars'),2).alias('Dollars'))
#     .alias('p')
#     .join(
#         _df_raw['InvoicePurchases']
#         .select('PONumber', 'PODate', 'VendorNumber','Quantity','Dollars','Approval','InvoiceDate','PayDate').withColumn('invoice', F.lit(True)).alias('i'),
#         common_cols,
#         'outer'
#     )
# )
# print('matching:', _df_temp.where('purchase and invoice').count())
# print('missing :', _df_temp.where('purchase is null or invoice is null').count())
# print('matching PODate:', _df_temp.where('p.PODate = i.PODate').count())
# print('matching VendorNumber:', _df_temp.where('p.VendorNumber = i.VendorNumber').count())
# print('matching InvoiceDate:', _df_temp.where('p.InvoiceDate = i.InvoiceDate').count())
# print('matching PayDate:', _df_temp.where('p.PayDate = i.PayDate').count())
# print('matching PODate AND VendorNumber:', _df_temp.where('p.PODate = i.PODate AND p.VendorNumber = i.VendorNumber').count())
# print('matching Dollars:', _df_temp.where('p.Dollars = i.Dollars').count())
# print('matching Quantity:', _df_temp.where('p.Quantity = i.Quantity').count())

# print('non-related rows')
# _df_temp.where('p.PODate != i.PODate or p.VendorNumber != i.VendorNumber').orderBy('p.PONumber').show(10,truncate=False)
# print('matching rows')
# _df_temp.where('p.PODate == i.PODate and p.VendorNumber == i.VendorNumber').orderBy('p.PONumber').show(10,truncate=False)


### WRITE TO DISK

In [18]:
dataframe_list = [
    '_df_product',
    '_df_vendor',
    '_df_city',
    '_df_store',
    '_df_inventory',
    '_df_purchase_order',
    '_df_invoice',
    '_df_purchase',
    '_df_sale',
]
for df_name in dataframe_list:
    print(f'Writing {df_name} to parquet...')
    globals()[df_name].write.mode('overwrite').parquet(f'./notebooks/parquet/{df_name.replace("_df_", "")}')
print('Finished!')

Writing _df_product to parquet...
Writing _df_vendor to parquet...
Writing _df_city to parquet...
Writing _df_store to parquet...
Writing _df_inventory to parquet...
Writing _df_purchase_order to parquet...
Writing _df_invoice to parquet...
Writing _df_purchase to parquet...
Writing _df_sale to parquet...
Finished!
